In [1]:
from modules.statistical import *

modules.preprocessing successfull loaded: 100%|███| 6/6 [00:03<00:00,  1.77it/s]


In [2]:
import numpy as np

In [3]:
layers = list(range(1,14))
layers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [4]:
alphas = [round(i,1) for i in np.arange(0.1,1.0,0.1)]
alphas.append(0.85)
#alphas = sorted(list(set(alphas)-set([0.2, 0.4])))
alphas

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.85]

# generation du tableau de comparaison de qualité de descripteurs dans la processus de selection

generate_g_b_impact_table(
    outputs_name='outputs_lts', 
    cwd=os.getcwd(), 
    outputPath='result_lts',
    layers=layers, 
    _type='qualitative',
    alphas=alphas
)

# Generation de tableau de comparaison de l'impact metriques des descripteurs

generate_report_tables_v3_2(
    outputs_name='outputs_lts', 
    cwd=os.getcwd(), 
    outputPath='result_lts',
    layers=layers, 
    glo=True,
    per=True,
    gap=True,
    logics=['GLO','PER'],#,'PER','GAP'
    approachs=['MlC','MCA'],
    configs=['MX','CX','CY','CXY'],#'MX',
    metrics=['accuracy','precision1','precision0','financial-cost'],#'accuracy','f1-score','precision1','recall1','f1-score1','precision0','recall0','f1-score0','financial-cost'
    metrics1=['precision1','financial-cost'],#'accuracy','f1-score','precision1','recall1','f1-score1','precision0','recall0','f1-score0','financial-cost'
    _type='qualitative',
    alphas=alphas,
    result_=[1],
    total_digits= None,
    decimal_digits= 0
)

a = {'precision1': {0.1: 0.5, 0.2: 0.6, 0.3: 1.0, 0.4: 0.7, 0.5: 0.7, 0.6: 0.5, 0.7: 0.5, 0.8: 0.8, 0.85: 0.8, 0.9: 6.8999999999999995}, 'financial-cost': {0.1: 4.0, 0.2: 2.7, 0.3: 1.1, 0.4: 1.5, 0.5: 3.5, 0.6: 1.5999999999999999, 0.7: 3.2, 0.8: 2.6, 0.85: 2.8000000000000003, 0.9: 2.4}}
a

# Generation de bar chart pour l'evaluation des descripteurs et du tableau recapitulatifs

generate_descriptor_ranking(
    outputs_name='outputs_lts', 
    cwd=os.getcwd(), 
    outputPath='result_lts',
    layers=layers, 
    _type='qualitative',
    alphas=alphas
)

# Proc Selection Analysis

In [5]:
import numpy as np

def vector_matching_precision(v1, v2, tolerance=0):
    """
    Calcule la précision de correspondance entre deux vecteurs.
    
    :param v1: Premier vecteur
    :param v2: Deuxième vecteur
    :param tolerance: Tolérance pour considérer deux valeurs comme correspondantes
    :return: Pourcentage de correspondance entre les vecteurs
    """
    if len(v1) != len(v2):
        raise ValueError("Les vecteurs doivent avoir la même longueur")
    
    v1, v2 = np.array(v1), np.array(v2)
    matches = np.abs(v1 - v2) <= tolerance
    precision = np.mean(matches) * 100
    
    return precision

In [6]:
def print_compare_feature_selection_protocole(
    store,
    output_path
):
    # add the resize box to ensure the scale of the table will be contain's inside the width space avalable.
    # start setting up the tabular dimensions setting
    table_header = """
        %\\begin{sidewaystable}
        \\resizebox{\\textwidth}{!}{

        \\begin{tabular}{|c|c|"""+("c|"*10)
    # setup information columns headears
    nbMCol =  10
    # add col for total results
    table_header+= "} "
    # add separator clines
    nb_cols = (2+nbMCol)
    table_header+= " \\cline{1-"+str(nb_cols)+"}" # corresponding to the number of columns

    # build the first line: metrics' line
    lines = ''
    # add the blank block
    lines += """
    \\multicolumn{2}{|c|}{}"""


    datasets = list(store.keys())
    methods = list(store[datasets[0]].keys())
    alphas = sorted(list({el for data in datasets for el in store[data][methods[0]].keys() }))
    # add alpha for metric
    for alpha in alphas:
        lines+= f" & {alpha}"
    # add the total name
    lines+= " \\\\ "
    lines+= " \\cline{1-"+str(nb_cols)+"""}
    """
    for folder in datasets:
        # fetch on model
        lines+= """
            \\multirow{3}{*}{"""+folder+"""}"""
        for mi, meth in enumerate(methods): 
            lines+= f""" & {meth}"""
            for ai, alpha in enumerate(alphas): # MlC, MCA
                lines+= f""" & {store[folder][meth][alpha] }""" if alpha in list(store[folder][meth].keys()) else " & "
            lines+= ("""\\\\ """+ """ \\cline{2-"""+str(nb_cols)+"""}

                    """) if mi != len(methods)-1 else ("""\\\\ """+ """ \\cline{1-"""+str(nb_cols)+"""}

                    """)
                

    lines+= """

    \\end{tabular}
    }
    %\\end{sidewaystable}"""

    table = table_header + lines
    create_domain(f"{output_path}/descriptComp/alpha/mlnGrowth1")
    filename1 = f"{output_path}/descriptComp/alpha/mlnGrowth1/mlnGrowth1.tex"
    _file = open(filename1, "w")
    _file.write(header+"""
                \\begin{table}[H]
                \\centering
                """+table+"""
                \\caption{default}
                \\label{default}
                \\end{table}"""+footer)
    _file.close()

In [7]:
records = joblib.load('./result_lts/best.tex')

# result structure
resultDict = {
    'dataset':[],
    'alpha': [],
    # 'QuartileThreshold':[],
    'elbowThreshold':[],
    'cumulative_difference_threshold':[],
    # 'variance_explained_threshold':[],
    'realThreshold':[]
}
res = {}
getTheBestAcc = lambda store, k: round(max([acc for layer,_,acc in store if k == layer]),4)

# walk on the datasets
for dataset in records.keys():
    # walk on alphas
    res[dataset] = {key:{} for key in ['CUSUM','Elbow','réel']}
    for alpha in records[dataset].keys():
        if len(records[dataset][alpha]['predicted_best_k']) != 0:
            resultDict['dataset'].append(dataset)
            resultDict['alpha'].append(alpha)
            # resultDict['QuartileThreshold'].append(getTheBestAcc(records[dataset][alpha]['list'],records[dataset][alpha]['predicted_best_k'][0]))
            elb = elbow_method(list(records[dataset][alpha]['accuracies'].values()))
            elb = elb if elb < max([layer for layer,_,_ in records[dataset][alpha]['list']]) else max([layer for layer,_,_ in records[dataset][alpha]['list']])
            resultDict['elbowThreshold'].append(getTheBestAcc(records[dataset][alpha]['list'],elb))
            res[dataset]['Elbow'][alpha] = getTheBestAcc(records[dataset][alpha]['list'],elb)
            cum = cumulative_difference_threshold(list(records[dataset][alpha]['accuracies'].values()))
            cum = cum if cum < max([layer for layer,_,_ in records[dataset][alpha]['list']]) else max([layer for layer,_,_ in records[dataset][alpha]['list']]) 
            resultDict['cumulative_difference_threshold'].append(getTheBestAcc(records[dataset][alpha]['list'],cum))
            res[dataset]['CUSUM'][alpha] = getTheBestAcc(records[dataset][alpha]['list'],cum)
            # resultDict['variance_explained_threshold'].append(getTheBestAcc(records[dataset][alpha]['list'],variance_explained_threshold(list(records[dataset][alpha]['accuracies'].values()))))
            resultDict['realThreshold'].append(round(max([acc for _,_,acc in records[dataset][alpha]['list']]),4) )
            res[dataset]['réel'][alpha] = round(max([acc for _,_,acc in records[dataset][alpha]['list']]),4)

dat = pd.DataFrame(resultDict)
print_compare_feature_selection_protocole(res,'result_lts')

Directory 'result_lts/descriptComp/alpha/mlnGrowth1' created successfully.


/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/modules/pipeline.py:5265: RuntimeWarning: invalid value encountered in scalar divide
  if cumulative_diff / total_diff >= threshold_percent:


In [8]:
# print(f"\nPrécision QuartileThreshold avec tolérance 0: {vector_matching_precision(resultDict['QuartileThreshold'], resultDict['realThreshold']):.2f}%")
print(f"\nPrécision elbowThreshold avec tolérance 1%: {vector_matching_precision(resultDict['elbowThreshold'], resultDict['realThreshold'],0.01):.2f}%")
print(f"\nPrécision cumulative_difference_threshold avec tolérance 1%: {vector_matching_precision(resultDict['cumulative_difference_threshold'], resultDict['realThreshold'], 0.01):.2f}%")
# print(f"\nPrécision variance_explained_threshold avec tolérance 0: {vector_matching_precision(resultDict['variance_explained_threshold'], resultDict['realThreshold']):.2f}%")


Précision elbowThreshold avec tolérance 1%: 70.27%

Précision cumulative_difference_threshold avec tolérance 1%: 64.86%
